<a href="https://colab.research.google.com/github/parkrye/Python/blob/main/202210_Bigdata/05_Reduction_Operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark import SparkConf, SparkContext

In [ ]:
conf = SparkConf().setMaster("local").setAppName("reduction-op")
sc = SparkContext(conf=conf)

# Reduce
- `RDD.reduce(<func>)`
- 사용자가 지정하는 함수를 받아 여러 개의 값을 하나로 줄여줍니다.

In [ ]:
from operator import add

In [ ]:
sample_rdd = sc.parallelize([1, 2, 3, 4, 5])
sample_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
sample_rdd.reduce(add)

15

**파티션에 따라 결과물이 달라집니다.**
- 분산된 파티션들의 연산과 합치는 부분을 나눠서 생각해야 합니다.

In [ ]:
sample_rdd = sc.parallelize([1, 2, 3, 4])
sample_rdd.reduce(lambda x, y : (x * 2)+ y)

26

In [ ]:
# 파티션을 1개로 지정
sc.parallelize([1, 2, 3, 4], 1).reduce(lambda x, y : (x * 2) + y)

26

In [ ]:
# 파티션을 2개로 지정
sc.parallelize([1, 2, 3, 4], 2).reduce(lambda x, y : (x * 2) + y)

18

In [ ]:
# 파티션을 3개로 지정
sc.parallelize([1, 2, 3, 4], 3).reduce(lambda x, y : ( x * 2 ) + y)

18

`[1]`, `[2]`, `[3,4]`



**파티션에 데이터가 하나만 있으면 reduce가 안됩니다.**

In [ ]:
sc.parallelize([1, 2, 3, 4], 4).reduce(lambda x, y : (x * 2 )+ y)

26

# Fold
- `RDD.fold(zeroValue, <func>)`
- `reduce`와 비슷하지만, `zeroValue`에 넣어놓고 싶은 시작값을 지정해서 `reduce`가 가능
    - `zeroValue`는 파티션 마다 계산이 일어날 때 하나씩 더해지는 값

In [ ]:
rdd = sc.parallelize([2, 3, 4], 4)

print(rdd.reduce(lambda x, y : (x * y))) # 2 x 3 -> x 4
print(rdd.fold(1, lambda x , y : (x * y))) # 1 x 2 x 3 x 4

24
24


In [ ]:
print(rdd.reduce(lambda x, y : x + y)) # 2 + 3 + 4
print(rdd.fold(1, lambda x, y : x + y)) #  (1 + 1) + ( 1 + 2 ) + ( 1 + 3 ) + ( 1 + 4 )
                                         #  빈 공간의 파티션에 숫자 1이 부여됩니다(zeroValue)

9
14


`fold`에서 시작값은 파티션 마다 부여

# GroupBy
- `RDD.groupBy(<func>)`
- 그룹핑 함수를 받아 `reduction`

In [ ]:
rdd = sc.parallelize([1, 1, 2, 3, 5, 8])
result = rdd.groupBy(lambda x : x % 2).collect()

sorted([(x, sorted(y)) for (x, y) in result])

[(0, [2, 8]), (1, [1, 1, 3, 5])]

# Aggregate(Action)
- `RDD.aggregate(zeroValue, seqOp, combOp)`
    - `zeroValue` : 각 파티션에서 누적할 시작 값
    - `seqOp` : 타입 변경 함수
        - 파티션 내에서 벌어지는 연산을 담당
    - `combOp` : 모든 결과를 하나로 합쳐주는 연산을 담당
- 파티션 단위의 연산 결과를 합쳐주는 과정을 거치게 된다.

In [ ]:
rdd = sc.parallelize([1, 2, 3, 4], 2)

seqOp = lambda x, y : (x[0] + y, x[1] + 1) # 파티션 내의 연산
combOp = (lambda x, y : (x[0] + y[0], x[1] + y[1])) # 파티션의 모든 결과를 최종 연산

print(rdd.aggregate((0, 0), seqOp, combOp))

(10, 4)


**연산 과정**

- partion 1
    - `x=[1, 2]`, `zeroValue=[0, 0]`
    - `x[0] = 0, x[1] = 0` ( zeroValue )
    - (`x[0] + y = 0 + 1`, `x[1] + 1 = 0 + 1`) -> (1, 1)
    - (`x[0] + y = 1 + 2`, `x[1] + 1 = 1 + 1`) -> **(3, 2)**
- partion 2
    * x = `[3, 4]`, zeroValue=`[0, 0]`
    * `x[0]=0, x[1]=0` ( zeroValue )
    * (`x[0] + y = 0 + 3`, `x[1] + 1 = 0 + 1`) ==> (3, 1)
    * (`x[0] + y = 3 + 4`, `x[1] + 1 = 1 + 1`) ==> **(7, 2)**

`combOp`를 이용해서 계산
- `x=(3, 2), y=(7, 2)`
- `(x[0] + y[0], x[1] + y[1]) = (10, 4)`

In [ ]:
sc.stop()